In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from flowdec.nb import utils as nbutils 
from flowdec import data as fd_data
# import pims
# import pims
from flowdec import restoration as fd_restoration
from flowdec import data as fd_data
from flowdec import psf as fd_psf
import dask
import dask.array as da
import tifffile as tf
from nd2reader import ND2Reader
# from pims import ND2_Reader 
# import aicsimageio.vendor.omexml as ome
import glob
import time, sys
sys.path.insert(0,'./libraries/')
from deco_libraries import update_progress, pimsmeta2OMEXML, Nd2meta2OMEXML, observer, init_RL_algo, deconv, depth_divide

In [2]:
#THINGS TO CHANGE!!!!! Always keep /*.nd2 at the end
#For gui make a function with all this input for verification purposes when loading the files
# dirname = "D:/Stephanie/20210203Livecaru/*nd2"
# dirname = "D:/JM/csbdeeptraining/GT/*nd2"
dirname = "/run/media/jmamede/Joao/CAruby/mellitin/*.nd2"


filelist = glob.glob(dirname)
filelist.sort()
# filelist.pop()
# filelist = filelist[2:3]

print(filelist)
print(filelist[0])
#Load the first file for reference and what's inside the file
frames = ND2Reader(filelist[0])
print(frames.sizes)
#Grab metadata
metadata = frames.metadata
print(metadata['channels'])

#If you don't have enough VRAM divide the XY to do the deconvolution that is automatically assembled after
xdivide = 1
ydivide = 1
depthdivide = depth_divide(xdivide, ydivide)

['/run/media/jmamede/Joao/CAruby/mellitin/iGFP_optiR3_050621_Metilin_RFP670.nd2', '/run/media/jmamede/Joao/CAruby/mellitin/iGFP_optiR3_050621_NOMetilin_RFP670 .nd2']
/run/media/jmamede/Joao/CAruby/mellitin/iGFP_optiR3_050621_Metilin_RFP670.nd2
{'x': 2048, 'y': 2048, 'c': 3, 't': 1, 'z': 9, 'v': 8}
['FITC', 'TRITC', 'Cy5']


In [3]:
# filelist.pop(1)
# filelist.pop(1)
# filelist
# test = filelist.pop(0)

In [4]:
#Need to make this a big more automatic, but indexing all the files and iterations is not easy
#In the version where the PSF is estimated by flowdec is easier because everything was set to automatic


PSFdir="/home/jmamede/Rush/PSF/"
# psf647 = tf.imread(PSFdir+"/AF647_0.3_EMCCD.tif")
# psfruby =  tf.imread(PSFdir+'/mRuby3_0.3_EMCCD.tif')
# psfgfp =  tf.imread(PSFdir+'/green_0.3_EMCCD.tif')
# psfdapi =  tf.imread(PSFdir+'/DAPI_0.3_EMCCD.tif')
# psf647 = tf.imread(PSFdir+"/AF647_0.5_EMCCD.tif")
# psfruby =  tf.imread(PSFdir+'/mRuby3_0.5_EMCCD.tif')
# psfgfp =  tf.imread(PSFdir+'/green_0.5_EMCCD.tif')
# psfdapi =  tf.imread(PSFdir+'/DAPI_0.5_EMCCD.tif')
psf647 = tf.imread(PSFdir+"/AF647.tif")
psfruby =  tf.imread(PSFdir+'/mRuby3.tif')
psfgfp =  tf.imread(PSFdir+'/green.tif')
psfdapi =  tf.imread(PSFdir+'/DAPI.tif')
# psfruby = psf647
# psfgfp = psf647
# psfdapi = psf647

#Clip Top and bottom, I found that if the PSF is calculated with distance from coverslip, it is flipped from Fiji
#As in, it is Top to Bottom, I acquire the images Bottom to Top
# remove = 1

remove = 13
psf647 = np.flip(psf647,
#                  axis=0
                )[remove:-remove,...]

psfgfp = np.flip(psfgfp,
#                  axis=0
                )[remove:-remove,...]
psfdapi = np.flip(psfdapi,
#                   axis=0
                 )[remove:-remove,...]
psfruby = np.flip(psfruby,
#                   axis=0
                 )[remove:-remove,...]

print(psf647.shape)
print(psfruby.shape)

(9, 256, 256)
(9, 256, 256)


In [5]:
algo =init_RL_algo(psfgfp.ndim
                   ,pad_mode='2357',pad_min=(2,12,12)
#                    ,pad_mode='log2',pad_min=(2,12,12)
#                    ,pad_mode='None',pad_min=(0,0,0)                   
                  )
print(filelist)
fname2 = filelist[1]
fname1 = filelist[0]
# filelist.pop(0)
print(filelist)

['/run/media/jmamede/Joao/CAruby/mellitin/iGFP_optiR3_050621_Metilin_RFP670.nd2', '/run/media/jmamede/Joao/CAruby/mellitin/iGFP_optiR3_050621_NOMetilin_RFP670 .nd2']
['/run/media/jmamede/Joao/CAruby/mellitin/iGFP_optiR3_050621_Metilin_RFP670.nd2', '/run/media/jmamede/Joao/CAruby/mellitin/iGFP_optiR3_050621_NOMetilin_RFP670 .nd2']


In [5]:

# frames.parser._raw_metadata.image_metadata
def get_deltaT(filename1,filename2):
    
    import pathlib

    fname1 = pathlib.Path(filename1)
    fname2 = pathlib.Path(filename2)
    import datetime
    mtime2 = datetime.datetime.fromtimestamp(fname2.stat().st_ctime)
    mtime1 = datetime.datetime.fromtimestamp(fname1.stat().st_ctime)
#     print(mtime2,mtime1)
#     print(fname2.stat().st_ctime - fname1.stat().st_ctime)
    return fname2.stat().st_ctime - fname1.stat().st_ctime
    # time_offset = 71146.03463482857
    
print(get_deltaT(fname1,fname2)/60)

time_offset = get_deltaT(fname1,fname2)

0.5287662347157797


In [6]:
frames =  ND2Reader(filelist[1])
# flip_channels = [-1]
# flip_channels = [63, 64, 65, 66, 67]
frames

<FramesSequenceND>
Axes: 6
Axis 'x' size: 1024
Axis 'y' size: 1024
Axis 'c' size: 3
Axis 't' size: 1
Axis 'z' size: 23
Axis 'v' size: 20
Pixel Datatype: <class 'numpy.float64'>

In [6]:
savePRJ = True
saveStack = False
# time_offset = 0
# maxT = 25
maxT = None
concattime = False
for fname in filelist:
    #to add the real time of experiment when we have to split it into different files
    if concattime:
#         time_offset = get_deltaT(filelist[0],fname)
        time_offset = get_deltaT(fname1,fname)
    else: time_offset = 0
    frames =  ND2Reader(fname)
    frames.iter_axes = 't'  # 't' is the default already
    frames.bundle_axes = 'zyx'  # when 'z' is available, this will be default

    channels = frames.metadata['channels']
    metadata = frames.metadata
    if saveStack:
        xml = Nd2meta2OMEXML(frames,time_offset=time_offset, maxT=maxT)
    if savePRJ:
        prjxml = Nd2meta2OMEXML(frames, project=True,time_offset=time_offset, maxT=maxT)

    chunk_size=(frames.sizes['z'],frames.sizes['y']//ydivide,frames.sizes['x']//xdivide)

    # frames.iter_axes = 'v'
    # for frame in frames:
    for visit in range(frames.sizes['v']):
#     for visit in range(21,31):

        frames.default_coords['v'] = visit
        stackfile = fname[:-4]+"_v"+str(visit+1).zfill(2)+'.ome.tiff'
        prjfile = fname[:-4]+"_v"+str(visit+1).zfill(2)+'_PRJ.ome.tiff'
        if saveStack:
            tifstack = tf.TiffWriter(stackfile, bigtiff=False,imagej=False)
            firststk = True
        if savePRJ:
            tifprj = tf.TiffWriter(prjfile, bigtiff=False, imagej=False)
            first = True
        if maxT == None:
            counterT = frames.sizes['t']
        else: counterT = maxT
            
            
        for time in range(counterT):
#         for time in range(25):

            res = np.zeros(
                (frames.sizes['z'],frames.sizes['c']
                 ,frames.sizes['y'],frames.sizes['x'])
                , dtype=np.float32)
            for i in range(frames.sizes['c']):
                frames.default_coords['c'] = i
#                     print('Channel:',metadata['channels'][i])
                ch = metadata['channels'][i]
                # ['a647', 'DAPI', 'GFP']
                # ['FITC', 'mRuby3', 'miRFP670']
                # ['FRET-gYFP-dsRED', 'mRuby', 'GFP', 'DAPI']
                # ['a647', 'DAPI', 'GFP']
                if ch == 'DAPI' or ch  == '470 nm':
                    psf = psfdapi
                elif ch == 'miRFP670' or ch == 'a647' or ch == 'mirfp670' or ch == 'farRED-EM':
                    psf = psf647
                elif ch == 'GFP' or ch == 'FITC'  or ch == 'GREEN-EM' or ch == 'FITC (Em)':
                    psf = psfgfp
                elif ch == 'mRuby3' or ch == 'mRuby' or ch ==  '555 nm' or ch == 'RED-EM':
                    psf = psfruby
                elif ch == 'FRET-gYFP-dsRED':
                    psf = psfruby
                else: psf = psfruby

                arr = da.from_array(frames[time]           
                                    , chunks=chunk_size)


                res[:,i,:,:] = da.map_overlap(deconv,arr, depth  = depthdivide ,boundary='reflect',
                                    dtype='float32', algo=algo,
                                    psf=psf, iters=100, use_ram=False
                                             ).compute(num_workers=1)

                # If we need to swap Channel and Z in the future
                #             img5d = np.swapaxes(img5d,0,1)

                update_progress( (visit+1) * (time+1) / (frames.sizes['v']* counterT) )
                print("Visit Point: {}/{} \t Time:{}/{} \t Channel:{}/{}".format(
                        visit+1, frames.sizes['v']
                        ,time+1,  counterT
                        , i+1, frames.sizes['c']))
# Make Function
            if saveStack:

                if firststk:
                    tifstack.save(res.astype(np.uint16),
                        compress='ZLIB',
                        description = xml.to_xml(),
                        photometric='minisblack',
                        metadata= None,
                        contiguous=False
                        )
                    firststk = False
                else:
                    tifstack.save(res.astype(np.uint16),
                        compress='ZLIB',
#                         description = xml.to_xml(),
#                         datetime= True,
                        photometric='minisblack',
                        metadata= None,
                        contiguous=False
                        )
            if savePRJ:
                if first:
                    tifprj.save(res.astype(np.uint16).max(axis=0),
                        compress='ZLIB',
                        description = prjxml.to_xml(),
                        photometric='minisblack',
                        #, datetime= True
                        metadata= None,
                        contiguous=False
                        )
                    first = False
                else:                  
                    tifprj.save(res.astype(np.uint16).max(axis=0),
                        compress='ZLIB',
#                         description = xml.to_xml(),
                        photometric='minisblack',
                        #, datetime= True
                        metadata= None,
                        contiguous=False,
                        )
        if saveStack:
            tifstack.close()
        if savePRJ:
            tifprj.close()

    update_progress(1)

Progress: [####################] 100.0%


In [15]:
print('done')
from numba import cuda
cuda.select_device(0)
cuda.close()
#the memory was released here!
cuda.select_device(0)

done


<weakproxy at 0x000001FE00E25368 to Device at 0x000001FE009A1BC8>

In [8]:
!echo 'buh'

buh


In [12]:
!/usr/bin/dbus-send --system --print-reply --dest="org.freedesktop.ConsoleKit" /org/freedesktop/ConsoleKit/Manager org.freedesktop.ConsoleKit.Manager.Stop

Error org.freedesktop.DBus.Error.ServiceUnknown: The name org.freedesktop.ConsoleKit was not provided by any .service files


In [11]:
import os
os.system("shutdown /s /")

256